In [1]:
from collections import Counter
from itertools import chain
from pandas import DataFrame
import nltk
from notebook_helper.process_data import *
import seaborn as sns
%matplotlib inline

In [2]:
PM1 = process_polymath("Polymath 1", split=True)

In [171]:
PM1_D_TREE = PM1['discussion threads', 'mthread (accumulated)'].iloc[-1].graph

In [172]:
corpus = {}
for node, data in PM1_D_TREE.nodes_iter(data=True):
    corpus[node] = data['com_content']

In [173]:
features = set(chain(*corpus.values()))
features = features.difference(nltk.corpus.stopwords.words('english'))
corpus_counted = {node: Counter(tokens) for node, tokens in corpus.items()}

In [28]:
data = DataFrame(index=corpus_counted.keys(),columns=sorted(list(features)), dtype='int')

In [29]:
for row in data.index:
    data.loc[row] = corpus_counted[row]

In [30]:
data = data.fillna(value=0)
data = data.astype('int')

In [63]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df=5)
vect.fit(corpus.values())

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [64]:
bagged = vect.transform(corpus)

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer

**to do:** install spacy package!

In [125]:
stemmer = nltk.stem.PorterStemmer
tfvect = TfidfVectorizer(min_df=2, ngram_range=(2, 8))
tf_corpus = tfvect.fit_transform(corpus.values())

In [126]:
import numpy as np
feature_names = np.array(tfvect.get_feature_names())
sorted_idf = np.argsort(tfvect.idf_)[::-1]
feature_names[sorted_idf[:150]]

array(['your idea', 'mathematical geniuses into two classes ordinary',
       'mathematical geniuses into two classes ordinary geniuses and',
       'mathematical ideas', 'mathematical problem', 'mathematician or',
       'mathematician to', 'mathematics is', 'mathematics one',
       'mathematics the', 'matter for', 'matter for those',
       'matter for those that', 'matter for those that are',
       'matter for those that are trying',
       'matter for those that are trying to',
       'matter for those that are trying to build',
       'buried in massive conversation', 'may end', 'may end up',
       'may end up being', 'may end up being wrong',
       'may end up being wrong foolish',
       'may end up being wrong foolish or',
       'may end up being wrong foolish or naive', 'buried in massive',
       'buried in',
       'build reputation how does being involved in discussion', 'may or',
       'may or may', 'may or may not',
       'mathematical geniuses into two classes ord

In [174]:
vect = CountVectorizer(max_features=10000, max_df=.15)
X = vect.fit_transform(corpus.values())

In [175]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_topics=20, learning_method="batch",
                                max_iter=25, random_state=0)

In [176]:
document_topics = lda.fit_transform(X)

In [177]:
lda.components_.shape

(20, 5546)

In [178]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vect.get_feature_names())

In [179]:
DataFrame(feature_names[sorting]).T.iloc[:5]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,wiki,their,above,project,solutions,paper,tier,statistics,class,idea,18,cells,paper,discussion,proof,http,type,thread,them,statistics
1,ve,credit,memory,ideas,gamma,their,paper,table,mathematics,question,slices,files,math,project,very,org,must,your,example,24
2,latex,who,arxiv,had,10,collaboration,been,pareto,number,who,slice,wordpress,affine,better,much,paper,slice,something,idea,score
3,com,polymath,well,polymath,solution,no,really,each,knol,no,11,pdf,tim,formula,mathematics,polymath1,cube,better,being,example
4,etc,question,use,who,values,contribute,etc,lookup,papers,density,adjacent,com,polymath,believe,because,title,center,comments,data,density
